In [1]:
print("Ok")

Ok


In [2]:
%pwd

'd:\\EPICS2.0\\AshaAI\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'd:\\EPICS2.0\\AshaAI'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents


In [7]:
extracted_data=load_pdf_file(data='Data/')

In [ ]:
# extracted_data

In [8]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [9]:
text_chunks=text_split(extracted_data)
print("Lengath of Text Chunks", len(text_chunks))

Lengath of Text Chunks 5860


In [10]:
# text_chunks


In [14]:
from langchain_huggingface import HuggingFaceEmbeddings

In [15]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings


In [16]:
embeddings = download_hugging_face_embeddings()